### Imports

In [1]:
import psycopg2
import sqlalchemy
import yaml

from sqlalchemy import create_engine
from sqlalchemy import inspect

import pandas as pd 

### read dbs creds

In [2]:
with open("../reference/country_data.yaml", "r") as db_creds_yaml:
    country_data = yaml.safe_load(db_creds_yaml)


FileNotFoundError: [Errno 2] No such file or directory: 'db_creds.yaml'

### initialize db enginge

In [ ]:
# Database connection details
# SQL Alchemy engine credentials
DATABASE_TYPE = 'postgresql'
DBAPI = 'psycopg2'
ENDPOINT = db_creds_dict["RDS_HOST"]
USER = db_creds_dict["RDS_USER"]
PASSWORD = db_creds_dict["RDS_PASSWORD"]
PORT = db_creds_dict["RDS_PORT"]
DATABASE = db_creds_dict["RDS_DATABASE"]

# Initialize SQL Alchemy engine
engine = create_engine(f"{DATABASE_TYPE}+{DBAPI}://{USER}:{PASSWORD}@{ENDPOINT}:{PORT}/{DATABASE}") 

print(inspect(engine).get_table_names())

['legacy_store_details', 'dim_card_details', 'legacy_users', 'orders_table']


In [ ]:
table_name = "legacy_users"

def read_rds_table(engine, table_name):
    db_table = pd.read_sql_table(table_name, engine)
    return db_table

db_table = read_rds_table(engine, table_name)
db_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15320 entries, 0 to 15319
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          15320 non-null  int64 
 1   first_name     15320 non-null  object
 2   last_name      15320 non-null  object
 3   date_of_birth  15320 non-null  object
 4   company        15320 non-null  object
 5   email_address  15320 non-null  object
 6   address        15320 non-null  object
 7   country        15320 non-null  object
 8   country_code   15320 non-null  object
 9   phone_number   15320 non-null  object
 10  join_date      15320 non-null  object
 11  user_uuid      15320 non-null  object
dtypes: int64(1), object(11)
memory usage: 1.4+ MB


### Processing...

In [ ]:
db_table.index = db_table.index.astype('int64')
db_table = db_table.sort_values('index')
db_table = db_table.drop('index', axis=1)
db_table = db_table.replace('NULL', None)
db_table = db_table.dropna(how='all')
mask = db_table["first_name"].str.contains('[0-9]+')
db_table = db_table[~mask]
db_table.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15284 entries, 0 to 15288
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   first_name     15284 non-null  object
 1   last_name      15284 non-null  object
 2   date_of_birth  15284 non-null  object
 3   company        15284 non-null  object
 4   email_address  15284 non-null  object
 5   address        15284 non-null  object
 6   country        15284 non-null  object
 7   country_code   15284 non-null  object
 8   phone_number   15284 non-null  object
 9   join_date      15284 non-null  object
 10  user_uuid      15284 non-null  object
dtypes: object(11)
memory usage: 1.4+ MB


In [ ]:
selected_country_codes = ['GB', 'DE', 'US']
mask = db_table.country_code.isin(selected_country_codes)

db_table[~mask]

,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid


In [ ]:
with open('country_codes.yaml', "r") as country_codes_file:
    country_codes_dict = yaml.safe_load(country_codes_file)

# print(country_codes_dict['United Kingdom'])

db_table['country_code'] = db_table['country'].replace(country_codes_dict)

In [ ]:
db_table.loc[9407]

first_name                                              Harriet
last_name                                                 Berry
date_of_birth                                        1943-12-29
company                                             Vincent LLC
email_address                               paige87@knight.info
address          Studio 5\nJamie village\nSouth Howard\nBB0 2QQ
country                                          United Kingdom
country_code                                                 GB
phone_number                                   +44(0)29 2018971
join_date                                            2000-12-25
user_uuid                  0f2b4d28-dcb9-45fe-aff3-7b9135f8d89b
Name: 9407, dtype: object

In [ ]:
db_table['country_code'].unique()

array(['DE', 'GB', 'US'], dtype=object)

In [5]:
codes = ['GB', 'DE', 'US']

for country in codes:
    print(codes.index(country))
    print(country)

0
GB
1
DE
2
US
